In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import time
from bs4 import BeautifulSoup
import os
import shutil
from datetime import datetime
import pypdf
import re
import os


def remove_files(download_path):
    for file in os.listdir(download_path):
        os.remove(os.path.join(download_path,file))
                  
def create_final_file(download_path, final_folder,user_name,date_name):
    for file in os.listdir(download_path):
        old_path = os.path.join(download_path,file)
        new_file = os.path.join(final_folder, date_name+user_name+".pdf")
        break
    shutil.copyfile(old_path, new_file)
    remove_files(download_path)
    return new_file
    
def get_timesheet(user_id,options,download_path,final_folder,date_name,user_name):
    remove_files(download_path)
    driver = webdriver.Chrome(options=options)
    driver.get("https://timesheet.cee.kworld.kpmg.com/")
    time.sleep(7)
    driver.find_element("id","SharingLabel").click()
    time.sleep(7)
    driver.switch_to.frame("contentFrame")
    time.sleep(15)
    tb_ids = driver.find_element("id","impersonateUserHolder")
    
    soup = BeautifulSoup(tb_ids.get_attribute("innerHTML"), 'html.parser')
    
    # Find all td elements
    td_elements = soup.find_all('td')
    
    # Print the text content of each td element
    for td in td_elements:
        if user_id in td.text:
            print(td.input['id'])
            print(tb_ids.find_element("id",td.input['id']).click())
    
    time.sleep(5)
    driver.find_element("id","impersonateUser").click()
    time.sleep(35)
    driver.switch_to.default_content()
    time.sleep(2)
    driver.switch_to.frame("contentFrame")
    time.sleep(2)
    period = driver.find_element("id","periodLabel").text
    if ((datetime.now().day >15) & ("16" in period))|((datetime.now().day <=15)& ("15" in period)):
        driver.find_element("id","previousPeriod").click()
        time.sleep(10)
        driver.switch_to.default_content()
        driver.switch_to.frame("contentFrame")

    time.sleep(7)
    driver.find_element("id","periodReportButton").click()
    time.sleep(7)
    
    #try:
        #dropdown = driver.find_element("xpath","//*[@id='reportDropDownList']/option[text()='Period summary']").click()
    #except Exception as E:
        #driver.find_element("id","previousPeriod").click()
        #time.sleep(4)
        #driver.switch_to.default_content()
        #driver.switch_to.frame("contentFrame")
    dropdown = driver.find_element("xpath","//*[@id='reportDropDownList']/option[text()='Period summary']").click()
    time.sleep(7)
    driver.switch_to.frame("reportFrame")
    time.sleep(7)
    driver.execute_script("$find('ReportViewerControl').exportReport('PDF');")
    time.sleep(7)
    driver.quit()
    return create_final_file(download_path, final_folder,user_name,date_name)

def check_pdf(pdf_path,user_id):
    pdf_file = pypdf.PdfReader(pdf_path, 'rb')
    text =""
    for nr_page in range(len(pdf_file.pages)):
        text = text+pdf_file.pages[nr_page].extract_text(extraction_mode="layout")
    charg = 0
    noncharg = 0
    okres = re.findall("\d{2}.\d{2}.\d{4} - \d{2}.\d{2}.\d{4}",text)[0]
    overtime = float(re.findall("(\d+,\d+)\n\s+TOIL",text)[0].replace(",","."))
    text_lines = text.split("\n")
    if user_id in text:
        for i,line in enumerate(text_lines):
            if "Total Chargeable" in line:
                charg = float(line.split(" ")[-1].replace(",","."))
            if "Total NonChargeable" in line:
                noncharg = float(line.split(" ")[-1].replace(",","."))
        return okres, charg, noncharg, overtime
    else:
        return "Nie znaleziono uzytkownika","",""

if __name__=="__main__":
    date_name = datetime.now().strftime("%y%m%d_")
    options= webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    download_path = os.path.join(os.getcwd(),"work_folder")
    
    final_folder = os.path.join(os.getcwd(),f"Final folder\\{date_name[:-1]}")
    if not os.path.exists(final_folder):
        os.mkdir(final_folder)
        
    
    options.add_argument('--headless=new')
    options.add_argument("--start-maximized")
    prefs = {
       "download.default_directory": download_path,
       "savefile.default_directory": download_path
    }
    
    options.add_experimental_option('prefs', prefs)
    
    #date_name = datetime.now().strftime("%y%m%d_")
    df = pd.read_excel("User_list.xlsx",na_filter=False,sheet_name="Rejestr")
    df['okres']="Nie znaleziono uzytkownika"
    df['Total Chargeable'] = 0
    df['Total NonChargeable'] = 0
    df['Overtime'] = 0
    for nr,row in df.iterrows():
        try:
            path_to_pdf = get_timesheet(row['Kod'],options,download_path,final_folder,date_name,row['Imie_nazwisko'])
            okres, TC,TNC,overtime = check_pdf(path_to_pdf,row['Kod'])
            df.loc[df['Kod']==row['Kod'],'okres']=okres
            df.loc[df['Kod']==row['Kod'],'Total Chargeable']=TC
            df.loc[df['Kod']==row['Kod'],'Total NonChargeable']=TNC
            df.loc[df['Kod']==row['Kod'],'Overtime']=overtime
        except Exception as E:
            print(E)
    df['Total']=df['Total Chargeable']+df['Total NonChargeable']
    writer = pd.ExcelWriter("User_list.xlsx", engine = 'openpyxl',mode='a',if_sheet_exists="replace")
    df[['Imie_nazwisko','Kod','okres','Total Chargeable','Total NonChargeable','Total','Overtime']].to_excel(writer,index=False,sheet_name=date_name[:-1])
    writer.close()

rblImpersonation_1
None


KeyboardInterrupt: 

In [16]:
### Poniższy skrypt zwraca liste osob, dla których posiadamy dostęp w timesheet

from selenium import webdriver
import time
from bs4 import BeautifulSoup

options= webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument('--headless=new')
driver = webdriver.Chrome(options=options)
driver.set_window_position(-100000,0)
driver.get("https://timesheet.cee.kworld.kpmg.com/")
time.sleep(1)
driver.refresh()
time.sleep(25)
driver.find_element("id","SharingLabel").click()
time.sleep(7)
driver.switch_to.frame("contentFrame")
time.sleep(5)
tb_ids = driver.find_element("id","impersonateUserHolder")   
soup = BeautifulSoup(tb_ids.get_attribute("innerHTML"), 'html.parser')
td_elements = soup.find_all('td')
for td in td_elements:
    tab_text  = td.text.split(" ")
    print(tab_text[1]+" "+tab_text[0] +"|"+tab_text[2][1:-1])
driver.quit()    


Bartłomiej Ekert|K9AB6
Łukasz Gordon|K9532
Antoni Szlafka|K99BU
Kacper Wardziak|K8710


In [36]:
from selenium import webdriver

options= webdriver.ChromeOptions()
options.add_argument('--headless=new')
#options.add_argument('disable-gpu')
#options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)
driver.minimize_window()
driver.get("https://stackoverflow.com/")

In [37]:
driver.quit()

In [42]:
import selenium
selenium.__version__

'4.12.0'